# Preparing Dataset

In [1]:
!gdown 119xWUEtRx5T92uThnpbMydY4Tz4HIycO

Downloading...
From (original): https://drive.google.com/uc?id=119xWUEtRx5T92uThnpbMydY4Tz4HIycO
From (redirected): https://drive.google.com/uc?id=119xWUEtRx5T92uThnpbMydY4Tz4HIycO&confirm=t&uuid=e0d97f1a-2180-4524-952e-0fa9f97aae88
To: /content/dataset.zip
100% 66.6M/66.6M [00:02<00:00, 23.0MB/s]


In [2]:
!unzip -q dataset.zip

In [3]:
import os
import pandas as pd

In [4]:
CODE_OUTPUT_PATH="output_code"
SOURCE_CODE_NAME="source<counter>.java"
code_dir = "code"
label_dir = "label"
prefix = 'dataset'
counter_code=0

In [5]:
def combine_and_save_source_code(folder_dir):
    global counter_code
    combined_contents = []
    for dir, _, files in os.walk(folder_dir):
        for file in files:
            file_path = os.path.join(dir, file)
            with open(file_path, 'r') as f:
                content = f.read()
                combined_contents.append(content)
    counter_code+=1
    output_path = os.path.join(CODE_OUTPUT_PATH,SOURCE_CODE_NAME.replace("<counter>",str(counter_code)))
    if not os.path.exists(CODE_OUTPUT_PATH):
        os.mkdir(CODE_OUTPUT_PATH)
    with open(output_path, 'w') as f:
        f.write("\n\n".join(combined_contents))
    return output_path

In [6]:
dataset = []
for dir, folders, files in os.walk(os.path.join(prefix,label_dir)):
    name = dir.split("\\")[-1]
    for file in files:
        last_name = None
        if file == 'designCodeSmells.csv' or file=='implementationCodeSmells.csv':
            file_path = os.path.join(dir, file)
            df = pd.read_csv(file_path)
            i=0
            while i < len(df):
              row=df.iloc[i]
              codesmells=[row[-1]]
              last_name=row[-2]
              for j in range(i+1,len(df)):
                row2=df.iloc[j]
                if row2[-2]!=last_name:
                  i=j-1
                  break
                codesmells.append(row2[-1])
              output_path = combine_and_save_source_code(os.path.join(prefix,code_dir,str(row[0]),str(row[0]),str(row[1]),str(row[2])))
              dataset.append({'file_path':output_path, 'codesmells':",".join(set(codesmells))})#,'dir':os.path.join(prefix,code_dir,str(row[0]),str(row[0]),str(row[1]),str(row[2]))
              i+=1

dataset = pd.DataFrame(dataset)
dataset.to_csv('dataset.csv', index=False)


In [7]:
!ls -1 output_code | wc -l # number of codes

39647


In [8]:
dataset.head()

,file_path,codesmells
0,output_code/source1.java,"Deficient Encapsulation,Insufficient Modulariz..."
1,output_code/source2.java,"Deficient Encapsulation,Insufficient Modulariz..."
2,output_code/source3.java,"Deficient Encapsulation,Insufficient Modulariz..."
3,output_code/source4.java,Insufficient Modularization
4,output_code/source5.java,Insufficient Modularization


In [9]:
!mkdir prepared_dataset
!mv dataset.csv prepared_dataset/dataset.csv
!mv output_code prepared_dataset
!zip -rq prepared_dataset.zip prepared_dataset

In [10]:
dataset.iloc[len(dataset) - 1]

file_path     output_code/source39647.java
codesmells              Empty catch clause
Name: 39646, dtype: object

In [11]:
len(dataset)

39647

In [12]:
df=dataset

In [13]:
ids = df["file_path"]

In [14]:
df[ids.isin(ids[ids.duplicated()])].sort_values("file_path")

,file_path,codesmells


In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
!cp prepared_dataset.zip drive/MyDrive/dataset/prepared_dataset.zip